In [5]:
import pandas as pd
import numpy as np
import os

In [14]:
def produceProbSummary(resultFolder):
    '''
    produce a summary of prediction probilities from a folder of srt files (csv format)
    input:
    
    resultFolder: string (folder in ../submissions/)
    
    output:
    
    results: pandas DataFrame containing:
        row_id
        pred0
        pred1
        pred2
        conf0
        conf1
        conf2
    
    
    '''
    resultFolder='../submissions/'+resultFolder
    
    results=[]
    for f in os.listdir(resultFolder):
        if f.endswith('.rst'):
            fi=resultFolder+'/'+f
            results.append(pd.read_csv(fi))
    results=pd.concat(results)
    results.drop(['accuracy','regionalFBScore'], inplace=True)
    results.sort_values(by=['originalIndex'], inplace=True)
    return results

In [15]:
produceProbSummary('knn-01').head()

,Unnamed: 0,accuracy,conf0,conf1,conf2,originalIndex,pred0,pred1,pred2,regionalFBScore,x,y
0,0,20.293838,0.316522,0.169611,0.159732,0,4393146716,8017323210,6131996960,-1,82.075,1333.584
0,0,15.440680,0.289022,0.089012,0.082195,1,2465239230,5801740503,9801651394,-1,3621.541,2479.498
0,0,17.923917,0.468894,0.288894,0.115907,2,2516481553,7862615088,7295979691,-1,3967.922,2300.354
0,0,17.923917,0.746153,0.110147,0.079010,3,7995458948,3243409743,8393706174,-1,489.510,1037.918
0,0,16.020600,0.368029,0.270289,0.148313,4,4764406629,8277155346,8711861736,-1,326.830,9530.892


In [19]:
def combineRecords(folders):
    print 'reading folders'
    results=[produceProbSummary(folder) for folder in folders]
    print 'reading is done.  Analyzing files'
    if len(list(set([len(r) for r in results])))!=1:
        print "unequal number of records in folders, aborting..."
        return
    
    places=['pred0', 'pred1', 'pred2']
    confs=['conf0', 'conf1', 'conf2']
    weights=[100, 10, 1]
    place_confs=zip(places, confs, weights)
    
    ranks=[]
    print 'making ranks'
    for i in xrange(len(results[0])):
        scores=dict()
        for r in results:
            rRec=r.iloc[i]
            for p, c, w in place_confs:
                if rRec[p] in scores:
                    scores[rRec[p]]+=rRec[c]*w
                else:
                    scores[rRec[p]]=rRec[c]*w
        scores=np.sort([[p,s] for p, s in scores], order='f1', axis=0)[-3:][::-1]
        ranks.append(' '.join([str(s[0]) for s in scores]))
        if i%1000000==0:
            print i, 'lines are done'
    
    res=pd.DataFrame({row_id: xrange(len(results[0])), 'place_id': ranks})
    print 'all done'
    return res

In [20]:
res=combineRecords(['knn-01', 'knn-02'])

reading folders
reading is done.  Analyzing files
making ranks


TypeError: 'numpy.float64' object is not iterable

In [18]:
res.head()

AttributeError: 'NoneType' object has no attribute 'head'